In [18]:
import sys
sys.path.append('/Users/AngelaLee_1/myenv/lib/python3.12/site-packages')
import abc_atlas_access
from pathlib import Path
from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache
download_base = Path('/Users/AngelaLee_1/myenv/lib/python3.12/site-packages')
abc_cache = AbcProjectCache.from_s3_cache(download_base)
feature_matrix_label = 'WMB-10Xv2-TH'
file = abc_cache.get_data_path(directory='WMB-10Xv2', file_name='WMB-10Xv2-TH/log2')

cell = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='cell_metadata',
    dtype={'cell_label': str}
)
cell.set_index('cell_label', inplace=True)
cluster_details = abc_cache.get_metadata_dataframe(
    directory='WMB-taxonomy',
    file_name='cluster_to_cluster_annotation_membership_pivoted',
    keep_default_na=False
)
cluster_details.set_index('cluster_alias', inplace=True)
cell_extended = cell.join(cluster_details, on='cluster_alias')

import anndata
adata = anndata.read_h5ad(file, backed='r')
gene_filtered = adata.var.iloc[:200, :] #Change the gene_filtered range 
asubset = adata[:, gene_filtered.index].to_memory()

In [8]:
#Gene expression data per cell
pred = (cell_extended['feature_matrix_label'] == feature_matrix_label)
cell_filtered = cell_extended[pred]
cell_filtered.head()

def create_expression_dataframe(ad, gf) :
    gdata = ad[:, gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join(gdata)
    return joined

def aggregate_by_metadata(df,gene_filtered, values, sort=False):
    grouped = df.groupby(values)[gene_filtered].mean()
    if sort:
        grouped = grouped.sort_values(by=gene_filtered[0], ascending=False)
    return grouped
ntexp = create_expression_dataframe(asubset, gene_filtered)
agg = aggregate_by_metadata(ntexp, gene_filtered.gene_symbol, values=['cell_label','cluster_alias'])
agg.head(5)

,,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37587,Gm37323,Mrpl15,Lypla1,...,Sema4c,D430040D24Rik,Fam178b,Cox5b,Actr1b,4933424G06Rik,Gm33533,Zap70,Tmem131,Gm38115
cell_label,cluster_alias,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACGAG-103_A01,5230,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,9.109224,0.000000,0.0,0.0,0.0,0.000000,0.0
AAACCTGAGAAACGCC-042_A01,5033,9.720479,6.732400,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,9.306012,6.732400,0.0,0.0,0.0,0.000000,0.0
AAACCTGAGAAAGTGG-040_B01,5023,7.767653,6.774258,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,10.221658,6.774258,0.0,0.0,0.0,6.774258,0.0
AAACCTGAGAAAGTGG-044_D01,5046,8.461960,0.000000,8.46196,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,7.466046,0.000000,0.0,0.0,0.0,0.000000,0.0
AAACCTGAGAACAATC-036_D01,5094,7.186204,8.181243,0.00000,0.0,0.0,0.0,0.0,0.0,8.764547,7.186204,...,0.0,0.0,0.0,8.181243,0.000000,0.0,0.0,0.0,0.000000,0.0


In [26]:
#Finding the mean gene expression
# Assuming you have already defined `cell_filtered`, `gene_filtered`, and `asubset` as per your previous context

# Define function to create expression dataframe
def create_expression_dataframe(ad, gf):
    gdata = ad[:, gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join(gdata)
    return joined

# Define function to aggregate by metadata
def aggregate_by_metadata(df, gene_symbols):
    grouped = df.groupby('cluster_alias')[gene_symbols].mean()
    return grouped

# Create expression dataframe
ntexp = create_expression_dataframe(asubset, gene_filtered)
agg_stats = aggregate_by_metadata(ntexp, gene_filtered.gene_symbol)

selected_cluster_aliases = [1745, 1790, 1794, 2200, 2301, 2303, 2304, 2309, 2310, 2444, 2879, 2881, 2884, 2887, 2895, 2898, 2899, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2910, 2911, 2912, 2913, 3425, 3432, 3434, 3490, 4486, 4517,
                    4687, 4690, 4702, 4708, 4709, 4721, 4728, 4729, 4730, 4731, 4732, 4735, 4737, 4770, 4837, 4839, 4840, 4844, 4846, 4851, 4852, 4856, 4858, 4861, 5031, 5035, 5036, 5038, 5039, 5044, 5048, 5049, 
                    5050, 5054, 5057, 5058, 5062, 5077, 5081, 5085, 5090, 5094, 5097, 5101, 5106, 5112, 5113, 5137, 5139, 5203, 5204, 5211, 5212, 5213, 5214, 5215, 5218, 5230, 5231, 5232, 5233, 5234, 5236, 5237, 5243, 5244, 5249, 5250, 5252, 5254, 5255, 
                    5261, 5262, 5268, 5274, 5283, 14865, 14930, 14956, 14962, 14977, 14980, 14983, 24467, 24472, 24473, 24574]  # Replace with your desired clusters
len(selected_cluster_aliases)
# Filter agg_stats to include only selected cluster_alias values
filtered_agg_stats = agg_stats.loc[selected_cluster_aliases]

# Print or display the filtered aggregated data
print(filtered_agg_stats)
filtered_agg_stats.to_csv('0-200_mean_v2_TH.csv', index=True)

                   Xkr4    Gm1992   Gm19938  Gm37381       Rp1  Sox17  \
cluster_alias                                                           
1745           7.947417  0.983436  2.049377      0.0  0.000000    0.0   
1790           8.948236  1.184351  3.435622      0.0  0.038038    0.0   
1794           9.588399  1.500440  4.507796      0.0  0.031238    0.0   
2200           8.257760  1.354365  3.131485      0.0  0.000000    0.0   
2301           7.639717  2.016544  2.172362      0.0  2.172362    0.0   
...                 ...       ...       ...      ...       ...    ...   
14983          0.000000  0.000000  0.000000      0.0  0.000000    0.0   
24467          7.884557  0.788447  2.662279      0.0  0.000000    0.0   
24472          7.735545  0.622895  1.843965      0.0  0.000000    0.0   
24473          8.582306  0.645123  2.987764      0.0  0.000000    0.0   
24574          8.214292  1.147497  3.189295      0.0  0.000000    0.0   

               Gm37587  Gm37323    Mrpl15    Lypla

In [20]:
#Finding the standard deviation
# Assuming you have already defined `cell_filtered`, `gene_filtered`, and `asubset` as per your previous context

# Define function to create expression dataframe
def create_expression_dataframe(ad, gf):
    gdata = ad[:, gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join(gdata)
    return joined

# Define function to aggregate by metadata using standard deviation
def aggregate_by_metadata_std(df, gene_symbols):
    grouped_std = df.groupby('cluster_alias')[gene_symbols].std()
    return grouped_std

# Create expression dataframe
ntexp = create_expression_dataframe(asubset, gene_filtered)

# Aggregate by metadata to get standard deviation
agg_stats_std = aggregate_by_metadata_std(ntexp, gene_filtered.gene_symbol)

selected_cluster_aliases = [1745, 1790, 1794, 2200, 2301, 2303, 2304, 2309, 2310, 2444, 2879, 2881, 2884, 2887, 2895, 2898, 2899, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2910, 2911, 2912, 2913, 3425, 3432, 3434, 3490, 4486, 4517,
                    4687, 4690, 4702, 4708, 4709, 4721, 4728, 4729, 4730, 4731, 4732, 4735, 4737, 4770, 4837, 4839, 4840, 4844, 4846, 4851, 4852, 4856, 4858, 4861, 5031, 5035, 5036, 5038, 5039, 5044, 5048, 5049, 
                    5050, 5054, 5057, 5058, 5062, 5077, 5081, 5085, 5090, 5094, 5097, 5101, 5106, 5112, 5113, 5137, 5139, 5203, 5204, 5211, 5212, 5213, 5214, 5215, 5218, 5230, 5231, 5232, 5233, 5234, 5236, 5237, 5243, 5244, 5249, 5250, 5252, 5254, 5255, 
                    5261, 5262, 5268, 5274, 5283, 14865, 14930, 14956, 14962, 14977, 14980, 14983, 24467, 24472, 24473, 24574]  # Replace with your desired clusters

# Filter agg_stats_std to include only selected cluster_alias values
filtered_agg_stats_std = agg_stats_std.loc[selected_cluster_aliases]

# Print or display the filtered aggregated data
print(filtered_agg_stats_std)

                   Xkr4    Gm1992   Gm19938  Gm37381       Rp1  Sox17  \
cluster_alias                                                           
1745           2.609710  2.562854  3.405516      0.0  0.000000    0.0   
1790           1.627426  2.582834  3.601335      0.0  0.504630    0.0   
1794           0.994686  2.810119  3.476700      0.0  0.492929    0.0   
2200           2.034112  2.631150  3.456773      0.0  0.000000    0.0   
2301           0.984192  3.492757  3.762641      0.0  3.762641    0.0   
...                 ...       ...       ...      ...       ...    ...   
14983               NaN       NaN       NaN      NaN       NaN    NaN   
24467          2.425629  2.209371  3.562544      0.0  0.000000    0.0   
24472          2.274024  2.011945  3.090787      0.0  0.000000    0.0   
24473          1.181611  2.028023  3.446927      0.0  0.000000    0.0   
24574          2.617324  2.645905  3.528707      0.0  0.000000    0.0   

               Gm37587  Gm37323    Mrpl15    Lypla

In [32]:
#Finding the n-value
# Assuming you have already defined `cell_filtered`, `gene_filtered`, and `asubset` as per your previous context

# Define function to create expression dataframe
def create_expression_dataframe(ad, gf):
    gdata = ad[:, gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join(gdata)
    return joined

# Define function to count number of cells in each cluster_alias for each gene
def count_cells_per_gene(df, gene_symbols):
    # Melting the dataframe to have a long format suitable for counting
    melted_df = df.melt(id_vars=['cluster_alias'], value_vars=gene_symbols, var_name='gene', value_name='expression')
    # Filtering out rows where expression is NaN
    melted_df = melted_df.dropna(subset=['expression'])
    # Counting the number of cells for each gene in each cluster_alias
    count_df = melted_df.groupby(['cluster_alias', 'gene']).size().reset_index(name='cell_count')
    return count_df

# Create expression dataframe
ntexp = create_expression_dataframe(asubset, gene_filtered)

# Get list of gene symbols
gene_symbols = gene_filtered.gene_symbol.tolist()

# Count number of cells in each cluster_alias for each gene
cell_counts = count_cells_per_gene(ntexp, gene_symbols)

# Filter cell counts to include only selected cluster_alias values
selected_cluster_aliases = [1745, 1790, 1794, 2200, 2301, 2303, 2304, 2309, 2310, 2444, 2879, 2881, 2884, 2887, 2895, 2898, 2899, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2910, 2911, 2912, 2913, 3425, 3432, 3434, 3490, 4486, 4517,
                    4687, 4690, 4702, 4708, 4709, 4721, 4728, 4729, 4730, 4731, 4732, 4735, 4737, 4770, 4837, 4839, 4840, 4844, 4846, 4851, 4852, 4856, 4858, 4861, 5031, 5035, 5036, 5038, 5039, 5044, 5048, 5049, 
                    5050, 5054, 5057, 5058, 5062, 5077, 5081, 5085, 5090, 5094, 5097, 5101, 5106, 5112, 5113, 5137, 5139, 5203, 5204, 5211, 5212, 5213, 5214, 5215, 5218, 5230, 5231, 5232, 5233, 5234, 5236, 5237, 5243, 5244, 5249, 5250, 5252, 5254, 5255, 
                    5261, 5262, 5268, 5274, 5283, 14865, 14930, 14956, 14962, 14977, 14980, 14983, 24467, 24472, 24473, 24574]  # Replace with your desired clusters

filtered_cell_counts = cell_counts[cell_counts['cluster_alias'].isin(selected_cluster_aliases)]

# Print or display the filtered cell counts
print(filtered_cell_counts)

        cluster_alias           gene  cell_count
27800            1745  0610009E02Rik         100
27801            1745  1700001O22Rik         100
27802            1745  1700007K13Rik         100
27803            1745  1700084E18Rik         100
27804            1745  1810012K08Rik         100
...               ...            ...         ...
186995          24574           Vav2          37
186996          24574          Wdr34          37
186997          24574           Wdr5          37
186998          24574        Zdhhc12          37
186999          24574           Zer1          37

[24600 rows x 3 columns]


In [56]:
import pandas as pd
import os

def find_gene_in_files(directory, gene):
    gene_data = []
    gene_files = []
    
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Adjust if your files have a different extension
            file_path = os.path.join(directory, filename)
            
            # Read the file
            df = pd.read_csv(file_path)
            
            # Check if the gene is in the columns
            if gene in df.columns:
                # Extract the data for the specific gene and the cluster_alias
                gene_values = df[[gene, 'cluster_alias']]
                
                # Add the original cluster_alias column from the file as the first column
                gene_values.insert(0, 'cluster_alias_original', df['cluster_alias'])
                
                gene_data.append(gene_values)
                gene_files.append(filename)
            else:
                print(f"Gene {gene} not found in file {filename}.")
    
    # Combine data from multiple files
    if gene_data:
        combined_data = pd.concat(gene_data, keys=gene_files, names=['Filename'])
        combined_data = combined_data.reset_index(level='Filename')
        return combined_data
    else:
        print(f"Gene {gene} not found in any files.")
        return None

# Example usage
directory = '/Users/AngelaLee_1/Downloads/Fos-Seq/10X TH Gene Mean'  # Replace with the path to your directory containing the CSV files
gene = 'Gm1992'  # Replace with the gene you are interested in

combined_gene_data = find_gene_in_files(directory, gene)

if combined_gene_data is not None:
    # Print or display the combined gene data
    print(combined_gene_data)


Gene Gm1992 not found in file 201-400_mean_v2_TH.csv.
                 Filename  cluster_alias_original    Gm1992  cluster_alias
0    0-200_mean_v2_TH.csv                    1745  0.983436           1745
1    0-200_mean_v2_TH.csv                    1790  1.184351           1790
2    0-200_mean_v2_TH.csv                    1794  1.500440           1794
3    0-200_mean_v2_TH.csv                    2200  1.354365           2200
4    0-200_mean_v2_TH.csv                    2301  2.016544           2301
..                    ...                     ...       ...            ...
118  0-200_mean_v2_TH.csv                   14983  0.000000          14983
119  0-200_mean_v2_TH.csv                   24467  0.788447          24467
120  0-200_mean_v2_TH.csv                   24472  0.622896          24472
121  0-200_mean_v2_TH.csv                   24473  0.645123          24473
122  0-200_mean_v2_TH.csv                   24574  1.147497          24574

[123 rows x 4 columns]
